This notebook preprocesses the data of all the New York Times that is going to be feed for the PCA and regression analysis.
The goal is to get a dataframe with:
- as data: the features of the speakers (year of birth date, nationality, gender, work occupation)
- as target: the number of occurrences

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import figure
import matplotlib.dates as mdates
import json

# **1. Loading Data**



Here are the paths to the data we use.

In [3]:
PATH_ROOT = '/content/drive/MyDrive/EPFL/ADA'
PATH_PARQUET = PATH_ROOT + '/Project datasets'
PATH_QUOTEBANK = PATH_ROOT + '/Quotebank'
PATH_TO_QUOTES = PATH_QUOTEBANK + '/quotes-{year}.json.bz2'
PATH_TO_CLEAN = PATH_ROOT + '/Cleaned_data'

##1.1 Loading quote topics

In [4]:
path_to_quotes = PATH_TO_CLEAN + '/quote_topics_cleaned.parquet'
df = pd.read_parquet(path_to_quotes)
df.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,topic
4,2020-01-23-024008,"He got on top of me, and he raped me.",Annabella Sciorra,Q231395,2020-01-23 00:00:00,75,733
5,2020-01-31-027972,How many players in the N.B.A. have a huge rol...,Brian Baldinger,Q4963033,2020-01-31 10:00:17,4,266
11,2020-02-21-029926,If we choose a candidate who appeals to a smal...,Michael R. Bloomberg,Q607,2020-02-21 01:07:54,4,10
12,2020-01-27-036296,"If your last name was not Biden, do you think ...",Joe Biden,Q6279,2020-01-27 14:33:49,14,85
13,2020-01-31-052530,"It was not manslaughter, it wasn't a crime,",Walter Van Steenbrugge,Q2104597,2020-01-31 17:57:18,2,214


In [5]:
df.info

<bound method DataFrame.info of                   quoteID  ... topic
4       2020-01-23-024008  ...   733
5       2020-01-31-027972  ...   266
11      2020-02-21-029926  ...    10
12      2020-01-27-036296  ...    85
13      2020-01-31-052530  ...   214
...                   ...  ...   ...
858354  2018-09-04-104135  ...  2027
858358  2018-06-11-107317  ...   754
858361  2018-06-14-137014  ...   578
858362  2018-08-29-137323  ...    48
858366  2018-09-29-072126  ...   616

[222649 rows x 7 columns]>

In [6]:
df.describe()

,numOccurrences,topic
count,222649.000000,222649.000000
mean,18.594245,1067.277585
std,109.800916,1412.439232
min,1.000000,0.000000
25%,1.000000,106.000000
50%,3.000000,474.000000
75%,7.000000,1441.000000
max,16362.000000,7071.000000


## 1.2. Loading of topics dataset

In [7]:
path_to_topics = '/content/drive/MyDrive/EPFL/ADA/topics_info.json'
df_topics = pd.read_json(path_to_topics, orient='index')
df_topics.head(22)

,Topic,Count,Name
0,-1,420063,-1_defendants_defendant_prosecution_dressed
1,0,4380,0_russias_putins_vladimir_moscow
2,1,3926,1_beijing_chinas_chinaus_beijings
3,2,2647,2_justices_judges_judicial_judiciary
4,3,2290,3_negro_africanamericans_blacks_africanamerican
5,4,2205,4_singing_sing_sang_songwriting
6,5,1980,5_yorkers_yorker_brooklyn_yorks
7,6,1942,6_solutions_fixing_fixes_repair
8,7,1904,7_cristiano_ronaldo_mourinho_zlatan
9,8,1791,8_deduction_taxation_deductions_taxed


## 1.3 Loading wikidata labels



Wiki quotations map the ID to their label. In that way, we know what the QID's mean.

In [8]:
wiki_quotation = pd.read_csv(PATH_PARQUET + '/wikidata_labels_descriptions_quotebank.csv.bz2', compression='bz2', index_col='QID')

We have a look at what it looks like:



In [9]:
wiki_quotation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36969 entries, Q31 to Q106376887
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Label        32951 non-null  object
 1   Description  24542 non-null  object
dtypes: object(2)
memory usage: 866.5+ KB


In [10]:
wiki_quotation.head()

,Label,Description
QID,,
Q31,Belgium,country in western Europe
Q45,Portugal,country in southwestern Europe
Q75,Internet,global system of connected computer networks
Q148,People's Republic of China,sovereign state in East Asia
Q155,Brazil,country in South America


## 1.4 Loading speakers dataset

We load the dataframe of the list of the speakers.

In [11]:
df_speakers = pd.read_parquet(PATH_TO_CLEAN + '/speakers_cleaned.parquet')

We have a look at what it looks like:

In [12]:
df_speakers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9055981 entries, 0 to 9055980
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   aliases             object 
 1   date_of_birth       float64
 2   nationality         object 
 3   gender              object 
 4   lastrevid           int64  
 5   ethnic_group        object 
 6   US_congress_bio_ID  object 
 7   occupation          object 
 8   party               object 
 9   academic_degree     object 
 10  id                  object 
 11  label               object 
 12  candidacy           object 
 13  type                object 
 14  religion            object 
 15  nb_aliases          int64  
dtypes: float64(1), int64(2), object(13)
memory usage: 1.1+ GB


In [13]:
df_speakers.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion,nb_aliases
0,"[Washington, President Washington, G. Washingt...",1732.0,"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443],5
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",1952.0,[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None,3
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",1868.0,[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None,2
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",1946.0,[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]",10
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",1599.0,[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None,120


In [14]:
df['speaker'].value_counts()

President Donald Trump     5137
President Trump            3622
Bill de Blasio              894
Hillary Clinton             788
Bernie Sanders              764
                           ... 
Alexandra Föderl-Schmid       1
red dragon                    1
Leandro Sosa                  1
Byron Dobell                  1
Ezra Edelman                  1
Name: speaker, Length: 46751, dtype: int64

#**2. Preprocessing**

##1. Creation of clean dataset: merge of quotebank with speakers

In [15]:
df_clean = pd.DataFrame()

df_clean = df[['quoteID','qids', 'topic', 'numOccurrences']].merge(df_speakers[['id', 'nationality', 'date_of_birth', 'gender', 'ethnic_group', 'occupation', 'party', 'religion']], how='left', left_on = 'qids', right_on='id')
df_clean.drop(['id'], axis=1, inplace =True)
df_clean.head()

,quoteID,qids,topic,numOccurrences,nationality,date_of_birth,gender,ethnic_group,occupation,party,religion
0,2020-01-23-024008,Q231395,733,75,[Q30],NaN,[Q6581072],None,"[Q3282637, Q10800557, Q10798782, Q2259451]",None,None
1,2020-01-31-027972,Q4963033,266,4,[Q30],1959.0,[Q6581097],None,[Q19204627],None,None
2,2020-02-21-029926,Q607,10,4,[Q30],1942.0,[Q6581097],[Q7325],"[Q43845, Q82955, Q2095549, Q36180, Q1424605, Q...","[Q29552, Q29468, Q29552]","[Q9268, Q1133485]"
3,2020-01-27-036296,Q6279,85,14,[Q30],1942.0,[Q6581097],None,"[Q82955, Q40348, Q1622272]",[Q29552],[Q1841]
4,2020-01-31-052530,Q2104597,214,2,[Q31],1964.0,[Q6581097],None,[Q40348],None,None


#**4. Quick Regression**

We create a dataset that is grouped by the id of the speakers and sums the number of occurrences of each quote of each speaker

In [16]:
df_raw = df[['quoteID','qids', 'numOccurrences']].copy()
df_raw = pd.DataFrame(df_raw.groupby('qids')['numOccurrences'].sum()).sort_values(by='numOccurrences', ascending=False)
df_raw.head()

,numOccurrences
qids,
Q22686,954029
Q6294,56371
Q167607,52039
Q76,47036
Q355522,31355


We merge the dataframe with the dataset of all the speakers to get their features

In [17]:
df_reg = df_raw.merge(df_speakers[['id', 'nationality', 'date_of_birth', 'gender', 'occupation']], how='left', left_on = 'qids', right_on='id')
df_reg.head()

,numOccurrences,id,nationality,date_of_birth,gender,occupation
0,954029,Q22686,[Q30],1946.0,[Q6581097],"[Q911554, Q557880, Q3427922, Q15980158, Q13152..."
1,56371,Q6294,[Q30],1947.0,[Q6581072],"[Q82955, Q40348, Q193391, Q36180, Q18814623, Q..."
2,52039,Q167607,[Q30],1960.0,[Q6581097],"[Q40348, Q1622272, Q82955]"
3,47036,Q76,[Q30],1961.0,[Q6581097],"[Q82955, Q40348, Q15958642, Q28532974, Q372436]"
4,31355,Q355522,[Q30],1942.0,[Q6581097],"[Q82955, Q40348, Q16533, Q1450482, Q211236]"


In [18]:
df_reg = df_reg.reset_index(drop=True)
df_reg.dropna(subset=['nationality', 'gender', 'date_of_birth', 'occupation'], inplace=True)
df_reg = df_reg.reset_index(drop=True)
df_reg['id'].value_counts().sum()

33623

The next three parts handle the list in gender, occupation and nationality. They split them in one hot columns to be ready for the PCA and regression

##1. Gender

In [19]:
gender_list = np.hstack(df_reg['gender'].to_list())
gender_list = gender_list[gender_list != None]
gender_list = np.unique(gender_list)

In [20]:
for gender in gender_list:
  df_reg['gender_'+gender]=0
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33623 entries, 0 to 33622
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   numOccurrences    33623 non-null  int64  
 1   id                33623 non-null  object 
 2   nationality       33623 non-null  object 
 3   date_of_birth     33623 non-null  float64
 4   gender            33623 non-null  object 
 5   occupation        33623 non-null  object 
 6   gender_Q1052281   33623 non-null  int64  
 7   gender_Q1097630   33623 non-null  int64  
 8   gender_Q12964198  33623 non-null  int64  
 9   gender_Q15145778  33623 non-null  int64  
 10  gender_Q15145779  33623 non-null  int64  
 11  gender_Q18116794  33623 non-null  int64  
 12  gender_Q189125    33623 non-null  int64  
 13  gender_Q1984232   33623 non-null  int64  
 14  gender_Q2449503   33623 non-null  int64  
 15  gender_Q27679766  33623 non-null  int64  
 16  gender_Q301702    33623 non-null  int64 

In [21]:
def calc_gender(row):
  for gender in row.gender:
    row['gender_'+gender] = 1
  return row

df_reg = df_reg.apply(calc_gender, axis=1)
df_reg.head()

,numOccurrences,id,nationality,date_of_birth,gender,occupation,gender_Q1052281,gender_Q1097630,gender_Q12964198,gender_Q15145778,gender_Q15145779,gender_Q18116794,gender_Q189125,gender_Q1984232,gender_Q2449503,gender_Q27679766,gender_Q301702,gender_Q48270,gender_Q6581072,gender_Q6581097,gender_Q859614
0,954029,Q22686,[Q30],1946.0,[Q6581097],"[Q911554, Q557880, Q3427922, Q15980158, Q13152...",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,56371,Q6294,[Q30],1947.0,[Q6581072],"[Q82955, Q40348, Q193391, Q36180, Q18814623, Q...",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,52039,Q167607,[Q30],1960.0,[Q6581097],"[Q40348, Q1622272, Q82955]",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,47036,Q76,[Q30],1961.0,[Q6581097],"[Q82955, Q40348, Q15958642, Q28532974, Q372436]",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,31355,Q355522,[Q30],1942.0,[Q6581097],"[Q82955, Q40348, Q16533, Q1450482, Q211236]",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


##2. Nationality

In [22]:
nat_list = np.hstack(df_reg['nationality'].to_list())
nat_list = nat_list[nat_list != None]
nat_list = np.unique(nat_list)

In [23]:
for nat in nat_list:
  df_reg['nat_'+nat]=0
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33623 entries, 0 to 33622
Columns: 320 entries, numOccurrences to nat_Q986
dtypes: float64(1), int64(315), object(4)
memory usage: 82.1+ MB


In [24]:
def calc_nat(row):
  for nat in row.nationality:
    row['nat_'+nat] = 1
  return row

df_reg = df_reg.apply(calc_nat, axis=1)
df_reg.head()

,numOccurrences,id,nationality,date_of_birth,gender,occupation,gender_Q1052281,gender_Q1097630,gender_Q12964198,gender_Q15145778,gender_Q15145779,gender_Q18116794,gender_Q189125,gender_Q1984232,gender_Q2449503,gender_Q27679766,gender_Q301702,gender_Q48270,gender_Q6581072,gender_Q6581097,gender_Q859614,nat_Q1000,nat_Q1005,nat_Q1006,nat_Q1008,nat_Q1009,nat_Q1011,nat_Q1014,nat_Q1016,nat_Q1019,nat_Q1025,nat_Q1027,nat_Q1028,nat_Q1029,nat_Q1030,nat_Q1032,nat_Q1033,nat_Q1036,nat_Q1037,nat_Q1039,...,nat_Q843,nat_Q844930,nat_Q846,nat_Q846570,nat_Q851,nat_Q853348,nat_Q854,nat_Q858,nat_Q862086,nat_Q8646,nat_Q865,nat_Q869,nat_Q8733,nat_Q873625,nat_Q878,nat_Q881,nat_Q884,nat_Q889,nat_Q890120,nat_Q902,nat_Q912,nat_Q916,nat_Q917,nat_Q924,nat_Q928,nat_Q929,nat_Q945,nat_Q948,nat_Q953,nat_Q954,nat_Q958,nat_Q96,nat_Q962,nat_Q963,nat_Q965,nat_Q967,nat_Q9676,nat_Q971,nat_Q974,nat_Q986
0,954029,Q22686,[Q30],1946.0,[Q6581097],"[Q911554, Q557880, Q3427922, Q15980158, Q13152...",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,56371,Q6294,[Q30],1947.0,[Q6581072],"[Q82955, Q40348, Q193391, Q36180, Q18814623, Q...",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,52039,Q167607,[Q30],1960.0,[Q6581097],"[Q40348, Q1622272, Q82955]",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,47036,Q76,[Q30],1961.0,[Q6581097],"[Q82955, Q40348, Q15958642, Q28532974, Q372436]",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,31355,Q355522,[Q30],1942.0,[Q6581097],"[Q82955, Q40348, Q16533, Q1450482, Q211236]",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##3. Occupation

In [25]:
occ_list = np.hstack(df_reg['occupation'].to_list())
occ_list = occ_list[occ_list != None]
occ_list = np.unique(occ_list)

In [26]:
for occ in occ_list:
  df_reg['occ_'+occ]=0
df_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33623 entries, 0 to 33622
Columns: 2136 entries, numOccurrences to occ_Q998628
dtypes: float64(1), int64(2131), object(4)
memory usage: 547.9+ MB


In [27]:
def calc_occ(row):
  for nat in row.occupation:
    row['occ_'+occ] = 1
  return row

df_reg = df_reg.apply(calc_occ, axis=1)
df_reg.head()

,numOccurrences,id,nationality,date_of_birth,gender,occupation,gender_Q1052281,gender_Q1097630,gender_Q12964198,gender_Q15145778,gender_Q15145779,gender_Q18116794,gender_Q189125,gender_Q1984232,gender_Q2449503,gender_Q27679766,gender_Q301702,gender_Q48270,gender_Q6581072,gender_Q6581097,gender_Q859614,nat_Q1000,nat_Q1005,nat_Q1006,nat_Q1008,nat_Q1009,nat_Q1011,nat_Q1014,nat_Q1016,nat_Q1019,nat_Q1025,nat_Q1027,nat_Q1028,nat_Q1029,nat_Q1030,nat_Q1032,nat_Q1033,nat_Q1036,nat_Q1037,nat_Q1039,...,occ_Q9340211,occ_Q9344260,occ_Q9352089,occ_Q935666,occ_Q9357633,occ_Q936969,occ_Q937857,occ_Q9379869,occ_Q9385011,occ_Q93878955,occ_Q9394993,occ_Q94084,occ_Q943995,occ_Q947873,occ_Q948329,occ_Q956365,occ_Q95721423,occ_Q957729,occ_Q96034777,occ_Q964475,occ_Q9648008,occ_Q970153,occ_Q97344615,occ_Q974144,occ_Q976015,occ_Q97767907,occ_Q97767991,occ_Q97768154,occ_Q97768158,occ_Q97768164,occ_Q97768167,occ_Q97768245,occ_Q97768274,occ_Q97768463,occ_Q978044,occ_Q98523757,occ_Q98686841,occ_Q98925420,occ_Q998550,occ_Q998628
0,954029,Q22686,[Q30],1946.0,[Q6581097],"[Q911554, Q557880, Q3427922, Q15980158, Q13152...",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,56371,Q6294,[Q30],1947.0,[Q6581072],"[Q82955, Q40348, Q193391, Q36180, Q18814623, Q...",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,52039,Q167607,[Q30],1960.0,[Q6581097],"[Q40348, Q1622272, Q82955]",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,47036,Q76,[Q30],1961.0,[Q6581097],"[Q82955, Q40348, Q15958642, Q28532974, Q372436]",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,31355,Q355522,[Q30],1942.0,[Q6581097],"[Q82955, Q40348, Q16533, Q1450482, Q211236]",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


##4. Final processing

In [28]:
df_reg.drop(['occupation', 'nationality', 'gender'], inplace=True, axis=1)

In [29]:
df_reg = df_reg.set_index('id')

In [30]:
count = 0
for col in df_reg.drop(['date_of_birth', 'numOccurrences'], axis=1).columns:
  #print(col)
  #print(col.split('_')[1])
  feature = wiki_quotation.filter(regex='{}$'.format(col.split('_')[1]), axis = 0)['Label']
  if len(feature) == 1 :
    #print(col)
    #print(type(col.split('_')[0]))
    #print(type(feature[0]))
    df_reg.rename(columns={col:(feature[0])},inplace=True)
  else:
    df_reg.drop([col], axis=1, inplace=True)
    count +=1
  #print(wiki_quotation.filter(regex='{}$'.format('gender_'+col), axis = 0)['Label'][0])
print(count)
df_reg.columns = df_reg.columns.fillna('to_drop')
df_reg.head()

1


,numOccurrences,date_of_birth,transgender female,intersex,genderqueer,cisgender male,cisgender female,genderfluid,transgender person,shemale,transgender male,transmasculine,two-spirit,non-binary,female,male,bigender,Gabon,The Gambia,Guinea,Ivory Coast,Cameroon,Cape Verde,Liberia,Libya,Madagascar,Mauritania,Mauritius,Morocco,Mozambique,Namibia,Niger,Nigeria,Uganda,Rwanda,São Tomé and Príncipe,Senegal,Seychelles,Sierra Leone,Somalia,...,speculator,associate professor,spy,make-up artist,game tester,nanny,association football player,lecturer,neurosurgeon,cultural entrepreneur,ice skater,visiting professor,impresario,television presenter,character actor,Liedermacher,discalced carmelite friar,photojournalist,HIV activist,cultural critic,banjoist,child actor,Methodist pastor,educator,basket weaver,professions libérales et assimilés,to_drop,cadres de la fonction publique,to_drop,corporate administrative and commercial executive,ingénieurs et cadres techniques d'entreprise,to_drop,anciens cadres,to_drop,executive,reggae singer,Storm chaser,KVN actor,bookseller,illuminator
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Q22686,954029,1946.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Q6294,56371,1947.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Q167607,52039,1960.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Q76,47036,1961.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Q355522,31355,1942.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [31]:
df_reg.columns = df_reg.columns.fillna('to_drop')
df_reg.drop('to_drop', axis = 1, inplace = True)
df_reg.head()

,numOccurrences,date_of_birth,transgender female,intersex,genderqueer,cisgender male,cisgender female,genderfluid,transgender person,shemale,transgender male,transmasculine,two-spirit,non-binary,female,male,bigender,Gabon,The Gambia,Guinea,Ivory Coast,Cameroon,Cape Verde,Liberia,Libya,Madagascar,Mauritania,Mauritius,Morocco,Mozambique,Namibia,Niger,Nigeria,Uganda,Rwanda,São Tomé and Príncipe,Senegal,Seychelles,Sierra Leone,Somalia,...,forensic psychologist,murderer,khatib,satirist,speculator,associate professor,spy,make-up artist,game tester,nanny,association football player,lecturer,neurosurgeon,cultural entrepreneur,ice skater,visiting professor,impresario,television presenter,character actor,Liedermacher,discalced carmelite friar,photojournalist,HIV activist,cultural critic,banjoist,child actor,Methodist pastor,educator,basket weaver,professions libérales et assimilés,cadres de la fonction publique,corporate administrative and commercial executive,ingénieurs et cadres techniques d'entreprise,anciens cadres,executive,reggae singer,Storm chaser,KVN actor,bookseller,illuminator
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Q22686,954029,1946.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Q6294,56371,1947.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Q167607,52039,1960.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Q76,47036,1961.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Q355522,31355,1942.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [32]:
df_reg = df_reg.loc[:,~df_reg.columns.duplicated()]

In [33]:
df_reg.to_parquet(PATH_TO_CLEAN + '/speakers_occurrence_new.parquet')